In [3]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 106.0 MB/s eta 0:00:0000:0100:01


In [4]:
import pennylane as qml
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.decomposition import PCA

n_qubits = 6
n_layers = 3

dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RY(x[i], wires=i)
        qml.RZ(x[i], wires=i)

def variational_ansatz(params):
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RY(params[l * n_qubits + i], wires=i)
            qml.CNOT(wires=[i, (i+1) % n_qubits])
            qml.RZ(params[l * n_qubits + i], wires=i)
        qml.Barrier()

@qml.qnode(dev)
def quantum_kernel(x1, x2, params):
    feature_map(x1)
    variational_ansatz(params)
    qml.adjoint(feature_map)(x2)
    return qml.expval(qml.PauliZ(0))

    
def cost(params, X):
    N = len(X)
    kernel_matrix = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            kernel_matrix[i, j] = float(quantum_kernel(X[i], X[j], params))  # Ensure it's a scalar value
    
    return np.sum(kernel_matrix)


In [ ]:
from scipy.optimize import minimize
from tqdm import tqdm

# Load MNIST dataset
(X_train, _), (_, _) = mnist.load_data()
X = X_train.reshape(-1, 28 * 28)  # Flatten images
X = PCA(n_components=n_qubits).fit_transform(X[:100])  # Reduce to n_qubits dimensions

# Cost function
def cost(params, X):
    N = len(X)
    kernel_matrix = np.zeros((N, N))
    
    for i in tqdm(range(N)):
        for j in range(N):
            kernel_matrix[i, j] = quantum_kernel(X[i], X[j], params)
    
    return np.sum(kernel_matrix)

params = np.random.rand(n_layers * n_qubits)

# Optimization
res = minimize(lambda p: cost(p, X), params.flatten(), method='COBYLA', options={'maxiter': 20})

 75%|███████▌  | 75/100 [01:41<00:33,  1.34s/it]

In [7]:
# from sklearn.cluster import KMeans

# # Compute quantum kernel matrix
# kernel_matrix = np.zeros((len(X), len(X)))
# for i in range(len(X)):
#     for j in range(len(X)):
#         kernel_matrix[i, j] = qml.math.toarray(quantum_kernel(X[i], X[j], params))

# # Apply k-means clustering to quantum feature space
# kmeans = KMeans(n_clusters=3)
# kmeans_labels = kmeans.fit_predict(kernel_matrix)

from sklearn.cluster import KMeans

# Compute quantum kernel matrix
kernel_matrix = np.zeros((len(X), len(X)))
for i in range(len(X)):
    for j in range(len(X)):
        kernel_matrix[i, j] = quantum_kernel(X[i], X[j], params)

# Apply k-means clustering to quantum feature space
kmeans = KMeans(n_clusters=3)
kmeans_labels = kmeans.fit_predict(kernel_matrix)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [11]:
from sklearn.metrics import silhouette_score

silhouette_classical = silhouette_score(X[:10], kmeans_labels)
silhouette_quantum = silhouette_score(kernel_matrix, kmeans_labels)

print("Silhouette Score (Classical):", silhouette_classical)
print("Silhouette Score (Quantum):", silhouette_quantum)

Silhouette Score (Classical): -0.1560289297267961
Silhouette Score (Quantum): 0.3223063633690929
